<a href="https://colab.research.google.com/github/MariaPetrovskaya/Data-Science-demo/blob/main/mashine%20learning/mashine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from joblib import dump
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

Импорт всех библиотек 

In [ ]:
data = pd.read_csv('/datasets/users_behavior.csv')
display (data.head(5))
print (data.info())


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


Открытие файла и изучение его.
**Вывод:** пропусков нет, названия столбцов и форматы понятны.  

## Разбейте данные на выборки

In [ ]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra'] 
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.40, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.50, random_state=12345)
print('features_train shape:', features_train.shape)
print('features_valid shape:', features_valid.shape)
print('features_test shape:', features_test.shape)
print('target_train shape:', target_train.shape)
print('target_valid shape:', target_valid.shape)
print('target_test shape:', target_test.shape)

features_train shape: (1928, 4)
features_valid shape: (643, 4)
features_test shape: (643, 4)
target_train shape: (1928,)
target_valid shape: (643,)
target_test shape: (643,)


В 2 этапа разделяю выборку: на обучающую и валид+тест, далее разделяю валидизирующую и тестовую выборки. В итоге разделение на выборки 60-20-20. 

## Исследуйте модели

In [ ]:
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944


Применила модель дерево с вариантами гиперпараметра 'количество веток'.
**Вывод:** accuracy_score самый высокий у модели с 3 ветками  


In [ ]:
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # обучите модель с заданным количеством деревьев
    model.fit(features_train, target_train) # обучите модель на тренировочной выборке
    result = model.score(features_valid,target_valid) # посчитайте качество модели на валидационной выборке
    print(model, '|', result)
    if result > best_result:
        best_model = model# сохраните наилучшую модель
        best_result = result#  сохраните наилучшее значение метрики accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print('Cамая эффективная модель - best_model',best_model)

RandomForestClassifier(n_estimators=1, random_state=12345) | 0.7107309486780715
RandomForestClassifier(n_estimators=2, random_state=12345) | 0.7636080870917574
RandomForestClassifier(n_estimators=3, random_state=12345) | 0.7387247278382582
RandomForestClassifier(n_estimators=4, random_state=12345) | 0.7713841368584758
RandomForestClassifier(n_estimators=5, random_state=12345) | 0.749611197511664
RandomForestClassifier(n_estimators=6, random_state=12345) | 0.7807153965785381
RandomForestClassifier(n_estimators=7, random_state=12345) | 0.7682737169517885
RandomForestClassifier(n_estimators=8, random_state=12345) | 0.7822706065318819
RandomForestClassifier(n_estimators=9, random_state=12345) | 0.7729393468118196
RandomForestClassifier(n_estimators=10, random_state=12345) | 0.7853810264385692
Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692
Cамая эффективная модель - best_model RandomForestClassifier(n_estimators=10, random_state=12345)


Попробовала модель Случайный лес с вариантами гиперпарамета "количество деревьев"
**Вывод**: Самая эффективная модель - 10 деревьев 

In [ ]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
accuracy = accuracy_score(target_valid, predictions_valid)
print("Accuracy модели логистической регрессии:", accuracy)
#dump(model, 'model_9_1.joblib')

Accuracy модели логистической регрессии: 0.7791601866251944


**Выводы** Accuracy модели логистической регрессии: 0.7791601866251944

**Выводы по разделу** Проведено сравнение трех типов моделей, в двух проведен подбор эффективных гиперпараметров. По критерию максимальной accuracy как лучший варинт выбраны 2 модели: дерево решений с 3 ветками (accuracy =0.785) и случайный лес с количеством деревьев = 10 (accuracy = 0.785)

## Проверьте модель на тестовой выборке

In [ ]:
model = DecisionTreeClassifier(random_state=12345,max_depth=3)
model.fit(features_train, target_train)
test_predictions = model.predict(features_test)
accuracy_score(target_test, test_predictions)
accuracy_test=accuracy_score(target_test, test_predictions)
# < напишите здесь код расчёта на тестовой выборке >
print("Accuracy")
print("Тестовая выборка:", accuracy_test) 

Accuracy
Тестовая выборка: 0.7791601866251944


На тестовой выборке применила модель Дерево решений, 3 веток.
**Вывод** На тестовой выборке результат Accuracy ниже чем, при обучении моделей (0.779), но достигнут уровень accuracy, требующийся в задании (не меньше 0.75). 

In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=10) # обучите модель с заданным количеством деревьев
model.fit(features_train, target_train)
test_predictions = model.predict(features_test)
accuracy_score(target_test, test_predictions)
accuracy_test=accuracy_score(target_test, test_predictions)
# < напишите здесь код расчёта на тестовой выборке >
print("Accuracy")
print("Тестовая выборка:", accuracy_test) 

Accuracy
Тестовая выборка: 0.7807153965785381


На тестовой выборке применила модель Случайный лес, 10 деревьев. 
**Выводы** Модель с 10 деревьев дает высокий результат 0.78

**Выводы по разделу** Обе выбранные модели - дерево с гиперпараметром 3 ветки, и случайный лес с гиперпараметров 10 деревьев на тестовой выборке дают высокий уровень accuracy (0.77, 0.78) 

## (бонус) Проверьте модели на адекватность

In [ ]:
model = DummyClassifier(strategy="most_frequent")
model.fit(features_train, target_train)
test_predictions = model.predict(features_test)
accuracy_score(target_test, test_predictions)
accuracy_test=accuracy_score(target_test, test_predictions)
# < напишите здесь код расчёта на тестовой выборке >
print("Accuracy")
print("Тестовая выборка алгоритм DummyClassifier:", accuracy_test) 

Accuracy
Тестовая выборка алгоритм DummyClassifier: 0.6842923794712286


**Выводы** "Наивный" алгорим DummyClassifier дает Accuracy ниже, чем наши модели, значит они способны предсказывать более точно. 

In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=8) # обучите модель с заданным количеством деревьев
model.fit(features, target)
predictions = model.predict(features)
accuracy_score(target, predictions)
accuracy=accuracy_score(target, predictions)
# < напишите здесь код расчёта на тестовой выборке >
print("Accuracy")
print("Общий датасет:", accuracy) 

Accuracy
Общий датасет: 0.9757311761045426


**Выводы** Accuracy на общем датасете выше, чем на выборках. 
    

In [ ]:
model = DecisionTreeClassifier(random_state=12345,max_depth=5)
model.fit(features, target)
predictions = model.predict(features)
accuracy_score(target, predictions)
accuracy =accuracy_score(target,predictions)
# < напишите здесь код расчёта на тестовой выборке >
print("Accuracy")
print("Общий датасет:", accuracy)

Accuracy
Общий датасет: 0.8170504044803982


**Вывод** Accuracy на общем датасете выше, чем на выборках.  

**Выводы** по вменяемости моделей: 1) проведено сравнение с моделью моделью DummyClassifier, accuracy предлагаемых моделей выше, чем у случайных алгоритмов. 2)Accuracy на общем датасете выше, чем на выборках - в этом случае переобучение. 